In [55]:
!pip install transformers datasets rouge langdetect sentence-transformers

import torch.nn.functional as F
import pandas as pd
import numpy as np
import os
import torch
import nltk
import pickle

from transformers import BertTokenizer
from torch.utils.data import random_split
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from torch.utils.data import DataLoader
from rouge import Rouge
from sklearn.cluster import KMeans

import sys
sys.path.append('/content/modules')

from modules.models import SummaryModel, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset

nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Package cess_esp is already up-to-date!
[nltk_data]    | Downloading packag

True

In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-german-cased")

data = pd.read_csv('/content/drive/MyDrive/export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()

val_d = data.iloc[50000:51500]

In [ ]:
#val_sentence_cluster = SentenceClusterDataset(tokenizer, d)
val_document_split = DocSplitDataset(tokenizer, d)

In [62]:
df_sentence_cluster = val_d
df_sentence_cluster['sentences'] = df_sentence_cluster['CONTENT'].apply(
    lambda x: nltk.sent_tokenize(
        x, language='german'))
df_sentence_cluster['sentences'] = df_sentence_cluster['sentences'].apply(
    lambda x: [
        s for s in x if len(
            s.split(' ')) > 4 and len(
            s.split(' ')) < 32])

sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

article_clusters = list()
with tqdm(df_sentence_cluster.iterrows(), total=df_sentence_cluster.shape[0], unit='steps') as tprogress:
  for i, _ in enumerate(tprogress):
    example_sentences = df_sentence_cluster['sentences'].iloc[i]
    sent_emb = sbert.encode(example_sentences)

    if len(sent_emb) > 0:
      num_sentences = len(example_sentences)
      num_clusters = int(num_sentences * 0.3)
      num_clusters = 1 if num_clusters == 0 else num_clusters
      clustering_model = KMeans(n_clusters=num_clusters)
      clustering_model.fit(sent_emb)
      cluster_assignment = clustering_model.labels_

      clustered_sentences = [[] for a in range(num_clusters)]
      for sentence_id, cluster_id in enumerate(cluster_assignment):
          clustered_sentences[cluster_id].append(example_sentences[sentence_id])
    
    article_clusters.append(clustered_sentences)

  df_sentence_cluster['clusters'] = article_clusters

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
 35%|███▌      | 529/1500 [00:43<00:46, 21.00steps/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (8). Possib

In [65]:
df_sentence_cluster[df_sentence_cluster['clusters'].str.len() == 1]

,CONTENT,sentences,clusters
52768,In der vergangenen Woche schrieb Annalena Baer...,[Dies ist eine für die Grünen-Chefin typische ...,[[Dies ist eine für die Grünen-Chefin typische...
52777,Man lässt ihm dort aus Respekt vor der Kultur ...,[Man lässt ihm dort aus Respekt vor der Kultur...,[[Man lässt ihm dort aus Respekt vor der Kultu...
52783,"Und wie hoch ist die Wahrscheinlichkeit, die z...","[Und wie hoch ist die Wahrscheinlichkeit, die ...","[[Und wie hoch ist die Wahrscheinlichkeit, die..."
52791,Es gibt im Fall des Erzbistums Köln Grund zum ...,[Es gibt im Fall des Erzbistums Köln Grund zum...,[[Es gibt im Fall des Erzbistums Köln Grund zu...
52792,Es gibt im Fall des Erzbistums Köln Grund zum ...,[Es gibt im Fall des Erzbistums Köln Grund zum...,[[Es gibt im Fall des Erzbistums Köln Grund zu...
...,...,...,...
54335,Leere Reihen im Impfzentrum der Messe Erfurt a...,[Leere Reihen im Impfzentrum der Messe Erfurt ...,[[Leere Reihen im Impfzentrum der Messe Erfurt...
54337,Es sind spektakuläre Aufnahmen. Die Nasa hat e...,[Die Nasa sprach von »sieben Minuten des Terro...,[[Die Nasa sprach von »sieben Minuten des Terr...
54341,»Minuteman III«-Rakete: Allzeit bereit im Beto...,[»Minuteman III«-Rakete: Allzeit bereit im Bet...,[[»Minuteman III«-Rakete: Allzeit bereit im Be...
54343,Die bislang wohl schlimmste Phase in der Coron...,[Die bislang wohl schlimmste Phase in der Coro...,[[Die bislang wohl schlimmste Phase in der Cor...


In [133]:
df_sentence_cluster['clusters'].str.len().sum()

9067

In [7]:
sent_cluster_checkpoint = torch.load('/content/drive/MyDrive/checkpoints/mobbs/sent-cluster/checkpoint-epoch-9')
doc_split_checkpoint = torch.load('/content/drive/MyDrive/checkpoints/mobbs/doc-split/checkpoint-epoch-8')

In [14]:
encoder_config_sc = ModelConfig(sent_cluster_checkpoint['encoder_config'])
decoder_config_sc = ModelConfig(sent_cluster_checkpoint['decoder_config'])

model_sc = SummaryModel(encoder_config_sc, decoder_config_sc)
model_sc.load_state_dict(sent_cluster_checkpoint['model_state_dict'])
model_sc.to('cuda:0')
model_sc.eval()

encoder_config_ds = ModelConfig(doc_split_checkpoint['encoder_config'])
decoder_config_ds = ModelConfig(doc_split_checkpoint['decoder_config'])

model_ds = SummaryModel(encoder_config_ds, decoder_config_ds)
model_ds.load_state_dict(doc_split_checkpoint['model_state_dict'])
model_ds.to('cuda:0')
model_ds.eval()
print('')

Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cl

## Document Split

In [20]:
# DocSplit
generated_summaries_ds = list()
ds_dl = DataLoader(val_document_split, batch_size=None)
with tqdm(ds_dl, unit='steps') as tprogress:
  for i, (x_enc, x_dec, targets) in enumerate(tprogress):
    summary = model_ds.summarize_sequences(x_enc)
    summary = tokenizer.decode(summary[0][0], skip_special_tokens=True)
    generated_summaries_ds.append(summary)

100%|██████████| 1448/1448 [22:40<00:00,  1.06steps/s]


In [134]:
val_document_split.df['split_doc'].str.len().sum()

7084

In [46]:
example_articles = list()
for i, v in enumerate(val_document_split.df['y_tokens']):
  example_articles.append(tokenizer.batch_decode(v, skip_special_tokens=True))

In [47]:
example_articles = list(map(lambda x: ' '.join(x), example_articles))

In [52]:
rouge = Rouge()
scores_document_split = rouge.get_scores(generated_summaries_ds, example_articles, avg=True)

In [53]:
scores_document_split

{'rouge-1': {'f': 0.1629610453149986,
  'p': 0.490648831451674,
  'r': 0.10554002584087839},
 'rouge-2': {'f': 0.027159911340215427,
  'p': 0.09604635321579469,
  'r': 0.017956003780484776},
 'rouge-l': {'f': 0.15137578364926713,
  'p': 0.45925730361908057,
  'r': 0.09780882299513118}}

## Sentence Cluster

In [81]:
df_sentence_cluster['clusters'] = df_sentence_cluster['clusters'].apply(lambda l: [x for x in l if len(x) > 0])
#df_sentence_cluster[df_sentence_cluster['clusters'].str.len() == 0]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [87]:
df_sentence_cluster.iloc[10]['clusters']

[['Die Architekten der neuen Weltunordnung nach 1990 haben die Zeit nicht genutzt zur historischen Lektüre.',
  'Beides zusammen ist noch niemals dauerhaft gelungen.'],
 ['Aber auch von einer globalen Sicherheitsarchitektur konnte nicht die Rede sein – jetzt und auf alle Zeiten.',
  'Das war nicht der ewige Frieden, aber immerhin eine gemeinsame Verständigung über die Gefahren und ihre Vermeidung.',
  'Es hätte ihnen deutlich werden müssen, dass das östliche Europa – alias „the bloodlands“ – quer durch die Jahrhunderte Gestaltungsprobleme aufwirft, die sich wiederholen und jedenfalls zu größtmöglicher Vorsicht raten.'],
 ['Russland sprach bald nur noch zu sich selbst.',
  'Zeile für Zeile beschreibt der frühere Kanzlerberater eine Situation, die längst hochgradig instabil ist: „Russisches Roulette.',
  'Oder will man den dramatischen, allumfassenden Aufstieg der chinesischen Weltmacht ignorieren, die neuen Bedrohungen aus Cyber-Space der Dynamik von Kräften überlassen, die immer schwie

In [92]:
example_articles = df_sentence_cluster['CONTENT'].to_list()

In [88]:
generated_summaries_sc = list()
with tqdm(df_sentence_cluster.iterrows(), total=df_sentence_cluster.shape[0], unit='steps') as tprogress:
  for i, _ in enumerate(tprogress):
    summary = ''
    summ_sentences = list()
    clustered_sentences = df_sentence_cluster.iloc[i]['clusters']
    for cluster in clustered_sentences:
        tok = tokenizer(cluster, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")
        tok_tensor = torch.tensor(tok['input_ids'].numpy()[:,1:]).to('cuda:0')
        summ_sentence = model_sc.summarize_sequences({'input_ids': tok_tensor})
        summ_sentence = tokenizer.decode(summ_sentence[0][0], skip_special_tokens=True)
        summ_sentences.append(summ_sentence)

    summary = ' '.join(summ_sentences)
    generated_summaries_sc.append(summary)

100%|██████████| 1500/1500 [13:48<00:00,  1.81steps/s]


In [93]:
scores_sentence_cluster = rouge.get_scores(generated_summaries_sc, example_articles, avg=True)

In [94]:
scores_sentence_cluster

{'rouge-1': {'f': 0.2053913658715149,
  'p': 0.4615740104993555,
  'r': 0.14561089731043703},
 'rouge-2': {'f': 0.06520293836373182,
  'p': 0.1453554090380121,
  'r': 0.0493056382326053},
 'rouge-l': {'f': 0.19565605135831216,
  'p': 0.4378374620594858,
  'r': 0.139114806737977}}

## Baselines

In [95]:
from transformers import EncoderDecoderModel, AutoTokenizer, AutoModelForSeq2SeqLM

model_bert = EncoderDecoderModel.from_pretrained('mrm8488/bert2bert_shared-german-finetuned-summarization').to('cuda:0')

tokenizer_t5 = AutoTokenizer.from_pretrained('deutsche-telekom/mt5-small-sum-de-en-v1')
model_t5 = AutoModelForSeq2SeqLM.from_pretrained('deutsche-telekom/mt5-small-sum-de-en-v1').to('cuda:0')

tokenizer_pegasus = AutoTokenizer.from_pretrained("google/pegasus-xsum")
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-xsum").to('cuda:0')

Downloading:   0%|          | 0.00/4.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


Downloading:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/653 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.12G [00:00<?, ?B/s]

In [ ]:
articles = val_d['CONTENT'].to_list()

In [129]:
bert_summaries = list()
with tqdm(articles, unit='steps') as tprogress:
  for i, article in enumerate(tprogress):
#for article in articles:
    bert_samples = tokenizer(article, return_tensors='pt', max_length=512, truncation=True)
    bert_input_ids = bert_samples['input_ids'].to('cuda:0')
    bert_attention_mask = bert_samples['attention_mask'].to('cuda:0')

    bert_summaries.append(tokenizer.decode(model_bert.generate(input_ids=bert_input_ids, attention_mask=bert_attention_mask)[0], skip_special_tokens=True))


100%|██████████| 1500/1500 [21:19<00:00,  1.17steps/s]


In [130]:
scores_bert_baseline = rouge.get_scores(bert_summaries, articles, avg=True)
scores_bert_baseline

{'rouge-1': {'f': 0.20365301431991176,
  'p': 0.8215358456337557,
  'r': 0.14140162679906468},
 'rouge-2': {'f': 0.14100502779954976,
  'p': 0.6343516427692202,
  'r': 0.10134751903668238},
 'rouge-l': {'f': 0.2021242559004764,
  'p': 0.8121906661894991,
  'r': 0.1405320486673659}}

{'rouge-1': {'f': 0.20365301431991176,
  'p': 0.8215358456337557,
  'r': 0.14140162679906468},
 'rouge-2': {'f': 0.14100502779954976,
  'p': 0.6343516427692202,
  'r': 0.10134751903668238},
 'rouge-l': {'f': 0.2021242559004764,
  'p': 0.8121906661894991,
  'r': 0.1405320486673659}}

In [115]:
t5_summaries = list()
with tqdm(articles, unit='steps') as tprogress:
  for i, article in enumerate(tprogress):
    t5_samples = tokenizer_t5(article, return_tensors='pt', truncation=True, max_length=512)
    t5_input_ids = t5_samples['input_ids'].to('cuda:0')
    t5_attention_mask = t5_samples['attention_mask'].to('cuda:0')

    t5_summaries.append(tokenizer_t5.decode(model_t5.generate(input_ids=t5_input_ids, attention_mask=t5_attention_mask)[0], skip_special_tokens=True))

100%|██████████| 1500/1500 [05:41<00:00,  4.39steps/s]


In [116]:
scores_t5_baseline = rouge.get_scores(t5_summaries, articles, avg=True)
scores_t5_baseline

{'rouge-1': {'f': 0.08749901360603406,
  'p': 0.8318751840751857,
  'r': 0.052037318468156464},
 'rouge-2': {'f': 0.04552448586303362,
  'p': 0.5801836089836093,
  'r': 0.027302936683364667},
 'rouge-l': {'f': 0.08570036419492887,
  'p': 0.8191340881340899,
  'r': 0.05090049238495527}}

In [117]:
pegasus_summaries = list()
with tqdm(articles, unit='steps') as tprogress:
  for i, article in enumerate(tprogress):
    pegasus_samples = tokenizer_pegasus(article, return_tensors='pt', truncation=True, max_length=512)
    pegasus_input_ids = pegasus_samples['input_ids'].to('cuda:0')
    pegasus_attention_mask = pegasus_samples['attention_mask'].to('cuda:0')

    pegasus_summaries.append(tokenizer_pegasus.decode(model_pegasus.generate(input_ids=pegasus_input_ids, attention_mask=pegasus_attention_mask)[0], skip_special_tokens=True))

100%|██████████| 1500/1500 [33:48<00:00,  1.35s/steps]


In [ ]:
del pegasus_summaries[880]
del pegasus_articles[880]

In [127]:
del pegasus_summaries[1432]
del pegasus_articles[1432]

In [128]:
scores_pegasus_baseline = rouge.get_scores(pegasus_summaries, pegasus_articles, avg=True)
scores_pegasus_baseline

{'rouge-1': {'f': 0.080290232761557,
  'p': 0.5945505794731156,
  'r': 0.04998674568393543},
 'rouge-2': {'f': 0.03656681604247566,
  'p': 0.2850480921782423,
  'r': 0.024273858722163713},
 'rouge-l': {'f': 0.07891418674528501,
  'p': 0.5835810601097782,
  'r': 0.049204889373036416}}

In [141]:
generated_summaries_ds.index(matching[0])

945

In [142]:
generated_summaries_sc[945]

'Mindestens in der Impfungen 12. März pro Woche sind die Zulassung kommen. Die Corona - Pandemie, die Hälfte eine Bevölkerungs sich insgesamt zu verhindern.d : Das Halbfinale der Polizei sind auch, voran Covid 19. im aus lassen ", trotz sie nicht weltweit in rund 22 Millionen Euro hatte. Für jüngere Personen wird es im Sommer wohl noch keine Impfung geben. Wir Coronaimpfung können vermutlich nicht und Juni ich ruft, der Leben war aus viele Klöck. Bislang wurden in Deutschland rund 3, 4 Millionen Sars - CoV - 2 - Infektionen nachgewiesen. Cundet zudem die Corona Mutante, wo ich gehen als ein Corona - Mobil vor allem der Menschen verdient, bevor mit den leer sind weitere Wochen. Die Coronaimpfung nicht und Covid und darf kann nur rund 10, um die Impfdosis seinen in der Klinik abwarten. hatte Nach der Pandemie der Impfstoffen da 18 Impfdot, jetzt aus Italien werden überversch ent gebracht worden. außerdem alle Meter. " Und in den kommenden Wochen werden die Zahlen weiter steigen, womöglic

In [144]:
len(generated_summaries_sc)

1500

In [162]:
generated_summaries_sc[949]

'Die Sieben - Tage - RKD - Gesamtzahl eines der oder mit Sars werden gehen? ( CDU, 43. 52. ). Die Corona - Pandemie ist - Programmdirektor die Impfung in beiden auch einst, dass Schwimm.. Die ehemalige Bundestagsfraktionschef Lothar Matthäus würde und von sich auch die Gesellschaft Probleme weniger, dass ein Umat hat. 2021 der Hauptstadt England : Union sagte Corona - Pandemiebiger ( M. Vor genau einer Woche waren es 344 Tote. Vor in der Regierung werde malige Hälfte geimpft seien, dass eine Studie. Polen aus Großbritannien liefern. " Das geht der EU am Dienstagmorgen in Deutschland den RKI, nachdem ab zwei 24. 00 Uhr pro Die Linke berichtet. Sie müsse, aber, die dass die Corona - Test bis Anfang Juli direkten. Land zu verlassen'

In [161]:
example_articles[949]

'Dieser Artikel wird fortlaufend aktualisiert.\nNichts mehr verpassen: Für die neuesten Entwicklungen und wichtigsten Hintergründe abonnieren Sie unseren Coronavirus-Newsletter.\n05.20 Uhr: Die Gesundheitsämter in Deutschland haben dem Robert Koch-Institut (RKI) binnen eines Tages 7534 Corona-Neuinfektionen gemeldet. Das geht aus Zahlen des RKI von Dienstagmorgen hervor, die den Stand des RKI-Dashboards von 05:08 Uhr wiedergeben. Zum Vergleich: Am Dienstag vor einer Woche hatte der Wert bei 10.976 gelegen. Die Zahl der binnen sieben Tagen gemeldeten Neuinfektionen pro 100.000 Einwohner lag laut RKI am Dienstagmorgen bundesweit bei 141,4 (Vortag: 146,9; Vorwoche: 167,6)\nDeutschlandweit wurden nach RKI-Angaben binnen 24 Stunden 315 neue Todesfälle verzeichnet. Vor genau einer Woche waren es 344 Tote.\nDas RKI zählte seit Beginn der Pandemie 3.433.516 nachgewiesene Infektionen mit Sars-CoV-2 in Deutschland. Die tatsächliche Gesamtzahl dürfte deutlich höher liegen, da viele Infektionen ni

In [156]:
val_document_split.df.iloc[945]['split_doc']

['Dieser Artikel wird laufend aktualisiert.Nichts mehr verpassen: Für die neuesten Entwicklungen und wichtigsten Hintergründe abonnieren Sie unseren Coronavirus-Newsletter.07.28 Uhr: Der Corona-Notstand für die Olympia-Stadt Tokio wird weniger als drei Monate vor Beginn der Sommerspiele abermals verlängert.Man plane eine Verlängerung bis Monatsende, erklärte der zuständige Minister Yasutoshi Nishimura.Die Verlängerung des inzwischen dritten Notstands bis zum 31. Mai gilt auch für die stark betroffene westliche Region Osaka.Darüber hinaus soll der Notstand auch auf die Präfekturen Aichi und Fukuoka ausgeweitet werden.Zwar hatte die Olympia-Stadt Tokio während der vergangenen »Goldenen Woche«, einer Aneinanderreihung nationaler Feiertage, relativ geringe Neuinfektionen gemeldet.Am Donnerstag waren es 591 Fälle innerhalb von 24 Stunden gewesen.Doch war wegen der Feiertage auch weniger getestet worden.Daher befürchten Experten, dass die Infektionszahlen nun wieder steigen dürften.Der Notst

In [164]:
corona_article = example_articles[949]
bert_samples = tokenizer(article, return_tensors='pt', max_length=512, truncation=True)
bert_input_ids = bert_samples['input_ids'].to('cuda:0')
bert_attention_mask = bert_samples['attention_mask'].to('cuda:0')

qual_summary = tokenizer.decode(model_bert.generate(input_ids=bert_input_ids, attention_mask=bert_attention_mask)[0], skip_special_tokens=True)

In [165]:
qual_summary

'Ihre Arbeit soll Einblick in die frühe Entwicklung des Menschen geben und Techniken zur künstlichen Befruchtung verbessern : Zwei internationale Forscherteams haben unabhängig voneinander im Labor menschliche Zellhaufen erzeugt, die als Modelle für frühe Embryonen sogenannte Blastozysten dienen können.'

In [166]:
corona_article

'Dieser Artikel wird fortlaufend aktualisiert.\nNichts mehr verpassen: Für die neuesten Entwicklungen und wichtigsten Hintergründe abonnieren Sie unseren Coronavirus-Newsletter.\n05.20 Uhr: Die Gesundheitsämter in Deutschland haben dem Robert Koch-Institut (RKI) binnen eines Tages 7534 Corona-Neuinfektionen gemeldet. Das geht aus Zahlen des RKI von Dienstagmorgen hervor, die den Stand des RKI-Dashboards von 05:08 Uhr wiedergeben. Zum Vergleich: Am Dienstag vor einer Woche hatte der Wert bei 10.976 gelegen. Die Zahl der binnen sieben Tagen gemeldeten Neuinfektionen pro 100.000 Einwohner lag laut RKI am Dienstagmorgen bundesweit bei 141,4 (Vortag: 146,9; Vorwoche: 167,6)\nDeutschlandweit wurden nach RKI-Angaben binnen 24 Stunden 315 neue Todesfälle verzeichnet. Vor genau einer Woche waren es 344 Tote.\nDas RKI zählte seit Beginn der Pandemie 3.433.516 nachgewiesene Infektionen mit Sars-CoV-2 in Deutschland. Die tatsächliche Gesamtzahl dürfte deutlich höher liegen, da viele Infektionen ni